In [1]:
%matplotlib inline
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge

from sklearn.metrics import r2_score

In [34]:
years = [2012, 2013, 2014, 2015, 2016]

## Preparing Weather Data

In [82]:
weather_data = pd.DataFrame()

for year in years:
    year_data = pd.read_csv('../data/{}.csv'.format(year), index_col=0)
    weather_data = pd.concat([weather_data, year_data])
    
weather_data.shape

(1824, 22)

In [83]:
weather_data.columns

Index(['stn', 'wban', 'yearmoda', 'temp', 'count_temp', 'dewp', 'count_dewp',
       'slp', 'count_slp', 'stp', 'count_stp', 'visib', 'count_visib', 'wdsp',
       'count_wdsp', 'mxspd', 'gust', 'max', 'min', 'prcp', 'sndp', 'frshtt'],
      dtype='object')

In [84]:
weather_data.head()

,stn,wban,yearmoda,temp,count_temp,dewp,count_dewp,slp,count_slp,stp,...,count_visib,wdsp,count_wdsp,mxspd,gust,max,min,prcp,sndp,frshtt
0,337610,99999,20120101,28.8,8,25.7,8,1019.2,8,996.1,...,8,4.4,8,5.8,999.9,33.8,24.8,0.00I,999.9,100000
1,337610,99999,20120102,30.2,8,26.0,8,1021.2,8,998.2,...,8,5.3,8,7.8,999.9,35.6,24.4,0.00I,999.9,0
2,337610,99999,20120103,34.3,8,31.1,8,1020.3,8,997.5,...,8,5.1,8,7.8,999.9,37.9,29.7,0.00I,999.9,0
3,337610,99999,20120104,35.9,8,35.8,8,1017.7,8,995.0,...,8,6.1,8,7.8,999.9,39.2,31.8,0.00I,999.9,100000
4,337610,99999,20120105,36.5,8,35.4,8,1005.9,8,983.5,...,8,6.1,8,11.7,999.9,41.9,32.2,99.99,999.9,110000


In [85]:
weather_data.drop(weather_data.columns[[0, 1,4,6,8,10,12,14,16]], axis=1, inplace=True)
weather_data['year'] = weather_data['yearmoda'] // 10000
weather_data['month'] = weather_data['yearmoda'] // 100 % 100
weather_data['day'] = weather_data['yearmoda'] % 100
weather_data['sndp'].replace(999.9,0, inplace = True)
weather_data['yearmoda1'] = weather_data['yearmoda'].astype(str).apply(lambda x: x[:9])
weather_data.index = pd.to_datetime(weather_data['yearmoda1'], format='%Y%m%d').values
weather_data.drop(weather_data.columns[[0,16]], axis=1, inplace=True)
for x in weather_data[weather_data['wdsp'] == 999.9].index:
    weather_data.loc[x,'wdsp'] = np.NaN
weather_data['wdsp'] = weather_data['wdsp'].interpolate(method='time')
for x in weather_data[weather_data['mxspd'] == 999.9].index:
    weather_data.loc[x,'mxspd'] = np.NaN
weather_data['mxspd'] = weather_data['mxspd'].interpolate(method='time')
for x in weather_data[weather_data['prcp'] == '99.99'].index:
    weather_data.loc[x,'prcp'] = np.NaN
for x in weather_data.index:
    if (str(weather_data.loc[x,'prcp'])[-1] == 'I') or (str(weather_data.loc[x,'prcp'])[-1] == 'H'):
        weather_data.loc[x,'prcp'] = np.NaN
for x in weather_data[weather_data['prcp'].notnull()].index:
    weather_data.loc[x,'prcp'] = weather_data.loc[x,'prcp'][:-1]
for x in weather_data.index:
    if (str(weather_data.loc[x,'max'])[-1] == '*'):
        weather_data.loc[x,'max'] = weather_data.loc[x,'max'][:-1]
    if (str(weather_data.loc[x,'min'])[-1] == '*'):
        weather_data.loc[x,'min'] = weather_data.loc[x,'min'][:-1]
weather_data['max']=weather_data['max'].astype(np.float64)
weather_data['min']=weather_data['min'].astype(np.float64)
weather_data['prcp']=weather_data['prcp'].astype(np.float64)
weather_data['prcp'] = weather_data['prcp'].interpolate(method='time')

In [86]:
weather_data['fog'] = weather_data['frshtt'] // 100000
weather_data['rain'] = weather_data['frshtt'] // 10000 % 10
weather_data['snow'] = weather_data['frshtt'] // 1000 % 10
weather_data['hail'] = weather_data['frshtt'] // 100 % 10
weather_data['thunder'] = weather_data['frshtt'] // 10 % 10
weather_data['tornado'] = weather_data['frshtt'] % 10
weather_data.drop(['frshtt'],axis = 1, inplace = True)

In [87]:
weather_data.head()

,temp,dewp,slp,stp,visib,wdsp,mxspd,max,min,prcp,sndp,year,month,day,fog,rain,snow,hail,thunder,tornado
2012-01-01,28.8,25.7,1019.2,996.1,6.4,4.4,5.8,33.8,24.8,NaN,0.0,2012,1,1,1,0,0,0,0,0
2012-01-02,30.2,26.0,1021.2,998.2,8.4,5.3,7.8,35.6,24.4,NaN,0.0,2012,1,2,0,0,0,0,0,0
2012-01-03,34.3,31.1,1020.3,997.5,5.8,5.1,7.8,37.9,29.7,NaN,0.0,2012,1,3,0,0,0,0,0,0
2012-01-04,35.9,35.8,1017.7,995.0,0.4,6.1,7.8,39.2,31.8,NaN,0.0,2012,1,4,1,0,0,0,0,0
2012-01-05,36.5,35.4,1005.9,983.5,3.1,6.1,11.7,41.9,32.2,NaN,0.0,2012,1,5,1,1,0,0,0,0


In [88]:
columns = [col for col in weather_data.columns if col not in ['month', 'day']]
weather_data = weather_data[columns]

In [89]:
weather_data['week'] = weather_data.index.week

In [90]:
weather_data_week = weather_data.groupby(['year', 'week']).mean()

In [92]:
weather_data_week.head()

temp       dewp          slp          stp      visib  \
year week                                                              
2012 1     33.975000  31.125000  1013.012500   990.325000   5.025000   
     2     30.142857  26.471429  1014.542857   991.642857   8.185714   
     3     24.685714  22.685714  1016.514286   993.285714   4.442857   
     4     13.242857   8.028571  1026.742857  1002.700000   8.300000   
     5      2.885714  -2.000000  1035.971429  1011.200000  11.328571   

               wdsp      mxspd        max        min      prcp      sndp  \
year week                                                                  
2012 1     6.275000   9.250000  37.850000  29.687500  0.174167  0.487500   
     2     6.242857   8.042857  34.471429  25.871429  0.053571  1.857143   
     3     4.442857   6.942857  28.500000  20.142857  0.184286  2.485714   
     4     8.500000  11.371429  17.428571   9.600000  0.067500  8.814286   
     5     8.085714  11.100000   8.028571  -2.800000  0.034643  7.785714   

                fog      rain      snow  hail  thunder  tornado  
year week                                                        
2012 1     0.250000  0.375000  0.250000   0.0      0.0      0.0  
     2     0.000000  0.142857  0.285714   0.0      0.0      0.0  
     3     0.142857  0.142857  0.857143   0.0      0.0      0.0  
     4     0.428571  0.000000  0.000000   0.0      0.0      0.0  
     5     0.000000  0.000000  0.285714   0.0      0.0      0.0

## Preparing Yield Data

In [194]:
def get_year_culture_week_ndvi():
    data = pd.DataFrame(columns = ['year', 'culture', 'week', 'ndvi', 'yields'])
    
    for year in years:
        df = pd.read_excel('../Сводная вегетация.xlsx', sheetname=str(year), header=1)

        ndvi_columns = [col for col in df.columns if 'неделя' in col]
        culture_column = 'Культура ' + str(year)
        yields_column = 'Урожайность, т./га.'
        interesting_columns = [culture_column], ndvi_columns, [yields_column]
        
        df = df[[culture_column] + ndvi_columns + [yields_column]]
        data_array = []
        
        for i in range(df.shape[0]):
            for j in range(1, df.shape[1] - 1):
                culture = df.iloc[i][culture_column]
                week = df.columns[j].replace('неделя ', '')
                ndvi = df.iloc[i, j]
                yields = df.iloc[i][yields_column]
                
                row = [year, culture, week, ndvi, yields]
                data_array.append(row)
                
        data_array = np.array(data_array)
        data_frame = pd.DataFrame(data_array, columns=data.columns)
        data = pd.concat([data, data_frame[data_frame['ndvi'] != 'nan']])
        
    return data

In [195]:
yield_data = get_year_culture_week_ndvi()
yield_data.head()

,year,culture,week,ndvi,yields
5,2012,Подсолнечник,48,0.382,2.7
6,2012,Подсолнечник,47,0.38,2.7
8,2012,Подсолнечник,45,0.353,2.7
10,2012,Подсолнечник,43,0.341,2.7
11,2012,Подсолнечник,42,0.354,2.7


In [197]:
yield_data.dtypes

year       object
culture    object
week       object
ndvi       object
yields     object
dtype: object

In [199]:
yield_data['year'] = yield_data['year'].astype(int)
yield_data['week'] = yield_data['week'].astype(int)
yield_data['ndvi'] = yield_data['ndvi'].astype(float)
yield_data['yields'] = yield_data['yields'].astype(float)

In [204]:
yield_data.dropna(inplace=True)

In [209]:
yield_data_culture = yield_data[yield_data['culture'] == 'Подсолнечник'][['year', 'week', 'ndvi', 'yields']]

In [210]:
yield_data_week = yield_data_culture.groupby(['year', 'week']).mean()

In [211]:
yield_data_week.head()

ndvi    yields
year week                    
2012 1     0.260000  0.550000
     11    0.209385  1.536923
     12    0.221154  1.536923
     14    0.238692  1.536923
     15    0.291154  1.536923

## Combining the datasets

In [212]:
data_week = pd.concat([weather_data_week, yield_data_week], axis=1)

In [213]:
data_week.head()

temp       dewp          slp          stp      visib  \
year week                                                              
2012 1     33.975000  31.125000  1013.012500   990.325000   5.025000   
     2     30.142857  26.471429  1014.542857   991.642857   8.185714   
     3     24.685714  22.685714  1016.514286   993.285714   4.442857   
     4     13.242857   8.028571  1026.742857  1002.700000   8.300000   
     5      2.885714  -2.000000  1035.971429  1011.200000  11.328571   

               wdsp      mxspd        max        min      prcp      sndp  \
year week                                                                  
2012 1     6.275000   9.250000  37.850000  29.687500  0.174167  0.487500   
     2     6.242857   8.042857  34.471429  25.871429  0.053571  1.857143   
     3     4.442857   6.942857  28.500000  20.142857  0.184286  2.485714   
     4     8.500000  11.371429  17.428571   9.600000  0.067500  8.814286   
     5     8.085714  11.100000   8.028571  -2.800000  0.034643  7.785714   

                fog      rain      snow  hail  thunder  tornado  ndvi  yields  
year week                                                                      
2012 1     0.250000  0.375000  0.250000   0.0      0.0      0.0  0.26    0.55  
     2     0.000000  0.142857  0.285714   0.0      0.0      0.0   NaN     NaN  
     3     0.142857  0.142857  0.857143   0.0      0.0      0.0   NaN     NaN  
     4     0.428571  0.000000  0.000000   0.0      0.0      0.0   NaN     NaN  
     5     0.000000  0.000000  0.285714   0.0      0.0      0.0   NaN     NaN

In [214]:
data_week.dropna(inplace=True)

In [215]:
data_week.reset_index(inplace=True)

In [216]:
data_week.head()

,year,week,temp,dewp,slp,stp,visib,wdsp,mxspd,max,...,prcp,sndp,fog,rain,snow,hail,thunder,tornado,ndvi,yields
0,2012,1,33.975000,31.125000,1013.012500,990.325000,5.025000,6.275000,9.250000,37.850000,...,0.174167,0.487500,0.250000,0.375000,0.25,0.0,0.000000,0.0,0.260000,0.550000
1,2012,11,38.257143,29.414286,1019.242857,996.600000,9.971429,5.457143,7.771429,45.957143,...,0.056000,2.971429,0.142857,0.000000,0.00,0.0,0.000000,0.0,0.209385,1.536923
2,2012,12,49.157143,33.028571,1023.242857,1000.971429,11.814286,6.857143,11.114286,59.671429,...,0.041143,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.221154,1.536923
3,2012,14,51.071429,39.271429,1008.200000,986.357143,8.914286,6.185714,9.742857,64.385714,...,0.075000,0.000000,0.285714,0.285714,0.00,0.0,0.142857,0.0,0.238692,1.536923
4,2012,15,46.700000,35.714286,1006.885714,984.871429,10.271429,6.800000,10.000000,57.542857,...,0.000000,0.000000,0.000000,0.142857,0.00,0.0,0.000000,0.0,0.291154,1.536923


In [217]:
data_week.shape

(162, 21)

## Train / test split

In [244]:
train_data = data_week[data_week['year'] != 2015]
test_data = data_week[data_week['year'] == 2015]

In [245]:
x_cols = list(data_week.columns.copy())
x_cols.remove('yields')

X_train = train_data[x_cols]
X_test = test_data[x_cols]

y_train = train_data['yields']
y_test = test_data['yields']

In [246]:
print('X_train:', X_train.shape)
print('X_test:', X_test.shape)
print('y_train:', y_train.shape)
print('y_test:', y_test.shape)

X_train: (115, 20)
X_test: (47, 20)
y_train: (115,)
y_test: (47,)


## Scaling input values

In [247]:
columns = X_train.columns
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

X_train = pd.DataFrame(X_train, columns=columns)
X_test = pd.DataFrame(X_test, columns=columns)

In [248]:
X_train.head()

,year,week,temp,dewp,slp,stp,visib,wdsp,mxspd,max,min,prcp,sndp,fog,rain,snow,hail,thunder,tornado,ndvi
0,0.0,0.000000,0.216946,0.304573,0.350117,0.328012,0.238569,0.521263,0.326441,0.165530,0.253644,0.196768,0.085741,0.333333,0.375000,0.4375,0.0,0.0,0.0,0.096982
1,0.0,0.196078,0.289665,0.266412,0.577384,0.571904,0.749263,0.373711,0.196742,0.290365,0.282763,0.063267,0.522613,0.190476,0.000000,0.0000,0.0,0.0,0.0,0.000000
2,0.0,0.215686,0.474770,0.347036,0.723293,0.741810,0.939528,0.626289,0.489975,0.501540,0.456836,0.046482,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.022550
3,0.0,0.254902,0.507278,0.486297,0.174570,0.173792,0.640118,0.505155,0.369674,0.574131,0.451741,0.084732,0.000000,0.380952,0.285714,0.0000,0.0,0.2,0.0,0.056155
4,0.0,0.274510,0.433042,0.406947,0.126628,0.116047,0.780236,0.615979,0.392231,0.468764,0.414945,0.000000,0.000000,0.000000,0.142857,0.0000,0.0,0.0,0.0,0.156674


In [249]:
X_test.head()

,year,week,temp,dewp,slp,stp,visib,wdsp,mxspd,max,min,prcp,sndp,fog,rain,snow,hail,thunder,tornado,ndvi
0,0.0,0.000000,0.167460,0.342932,0.540741,0.533140,0.000000,0.388451,0.233509,0.076213,0.259677,0.031746,0.166667,1.000000,0.833333,0.333333,0.0,0.0,0.0,0.046149
1,0.0,0.020833,0.171958,0.317128,0.018803,0.012209,0.213580,0.574803,0.412929,0.109822,0.196891,0.130291,0.583333,0.666667,0.666667,0.333333,0.0,0.0,0.0,0.000000
2,0.0,0.041667,0.132540,0.204562,0.197151,0.183140,0.564198,0.380577,0.180739,0.084497,0.169765,0.079365,0.000000,0.166667,0.500000,0.333333,0.0,0.0,0.0,0.003377
3,0.0,0.062500,0.000000,0.000000,0.687179,0.659302,0.524691,0.435696,0.308707,0.000000,0.000000,0.010582,1.000000,0.500000,0.000000,0.333333,0.0,0.0,0.0,0.008382
4,0.0,0.104167,0.282011,0.429693,0.420513,0.423837,0.437037,0.280840,0.129288,0.226509,0.331911,0.073082,0.000000,0.500000,0.500000,0.000000,0.0,0.0,0.0,0.045914


## Model training

In [251]:
def r2_of(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return r2_score(y_test, y_pred)

In [252]:
r2_of(RandomForestRegressor())

-77.071734238912995

In [257]:
r2_of(LinearRegression())

-10.830319426518049

In [258]:
r2_of(SVR())

-5.0273317243568085